In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd 
import bs4 as bs4
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
import urllib
from PIL import Image


In [6]:
Hon = pd.read_excel(r'/Users/krichnorakhun/Documents/GP/inventory/List of Required EPC/Item.xlsx')

In [7]:
Hon = Hon[['PartNo']].drop_duplicates()
Hon = Hon.reset_index().drop('index',axis=1)

In [352]:
driver = webdriver.Chrome()
driver.get('https://hatcepc.honda.co.th/epc/index.jsf')
userInput = driver.find_element(By.ID,'loginForm:j_idt19')
userInput.send_keys('AD009252')
pwdInput = driver.find_element(By.ID,'loginForm:password1')
pwdInput.send_keys('Nutty@012')
pwdInput.send_keys(Keys.ENTER)

In [5]:
def SetupConfig():
    driver.find_element(By.XPATH,'/html/body/div[1]/div/span/form/div[1]/div[3]').click()
    driver.find_element(By.XPATH,'/html/body/div[12]/div/ul/li[2]').click()
    driver.find_element(By.XPATH,'/html/body/div[2]/form[1]/ul/li[4]/span/div[1]/div[2]/ul/li[3]').click()

In [6]:
def InputSearchBox(PartNo):             
        wait = WebDriverWait(driver, 10)  # Adjust the timeout as needed
        wait.until(EC.element_to_be_clickable((By.ID, 'searchValue')))        
        driver.find_element(By.ID,'searchValue').clear()
        driver.find_element(By.ID,'searchValue').send_keys(PartNo)
        driver.find_element(By.ID,'searchValue').send_keys(Keys.ENTER)
        waiting()
        print('input '+PartNo)

In [7]:
data = driver.page_source
soup = bs4.BeautifulSoup(data)

In [8]:
foundItem = len(soup.find_all('tbody',{'id':'itemSearchTable_data'})[0].find_all('tr'))

In [7]:
def ChangeDisplayItem():
    wait = WebDriverWait(driver, 10)
    try:
        wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/form[2]/div/fieldset/div/div/div[3]/select")))
        data = driver.page_source
        soup = bs4.BeautifulSoup(data)

        #To Select 10 rows per page.
        optionDisplay = soup.find_all('select')[3].find_all('option')
        optionDisplay = [x.text for x in optionDisplay]
        optionDisplay ='50' if '50' in optionDisplay else max(optionDisplay, key=int)
        optionDisplay
        dropdownMaxDisplay = driver.find_element(By.XPATH,'/html/body/div[2]/form[2]/div/fieldset/div/div/div[3]/select')
        # Create a Select object
        select = Select(dropdownMaxDisplay)
        select.select_by_visible_text(optionDisplay)
        waitingNextPage()
        data = driver.page_source
        soup = bs4.BeautifulSoup(data)
        foundPage = driver.find_element(By.XPATH,'/html/body/div[2]/form[2]/div/fieldset/div/div/div[3]/span[1]').text
        foundPage = int(foundPage.split(' ')[2].replace(')',''))
        return(foundPage)

    except TimeoutException:
        data = driver.page_source
        soup = bs4.BeautifulSoup(data)
        foundItem = len(soup.find_all('tbody',{'id':'itemSearchTable_data'})[0].find_all('tr'))
        if foundItem <= 5:
            return(1)           

In [10]:
data = driver.page_source
soup = bs4.BeautifulSoup(data)

In [ ]:
foundItem = driver.find_element(By.XPATH,'/html/body/div[2]/form[2]/div/fieldset/div/div/div[3]/span[1]').text
foundItem = int(foundItem.split(' ')[2].replace(')',''))

In [8]:
def findNLoop():
    wait = WebDriverWait(driver, 10)
    wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/form[2]/div/fieldset/div/div/div[3]/select")))
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    nLoop = len(soup.find_all('span',{'class':'ui-paginator-pages'})[0].find_all('a'))
    return(nLoop)

In [9]:
def ClosePartList():
    driver.find_element(By.XPATH,'/html/body/div[2]/form[2]/div/fieldset/div/div/div[1]/button/span[1]').click()
    return

In [10]:
def getModel():
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    wait = WebDriverWait(driver, 30) 
    try:
        wait.until(EC.presence_of_element_located((By.ID, 'searchResultPanel')))
        MatchPartList = soup.find_all('tbody',{'id':'itemSearchTable_data'})[0].find_all('tr')
        part_list = [x.find_all('td')[0].text for x in MatchPartList]
        part_desc_list = [x.find_all('td')[1].text.replace(',',':') for x in MatchPartList]
        # Combine the two lists into a list of tuples
        combined_data = list(zip(part_list, part_desc_list))
        
        MatchPartList = pd.DataFrame(combined_data, columns=['Part', 'PartDescription'])
        MatchPartList['PartID'] = MatchPartList['Part'].apply(lambda x : x.split('(',1)[0].split(' - ')[0])
        MatchPartList['CarModel'] = MatchPartList['Part'].apply(lambda x : x.split('(',1)[1].split(')')[0] )
        MatchPartList = MatchPartList.loc[MatchPartList['CarModel'].apply(lambda x : x.find(' TH '))>=0]
    except TimeoutException:
        print("Timeout occurred! Performing alternative steps.")
    finally:
        print('Got:'+str(len(MatchPartList)))
    return(MatchPartList)

In [11]:
def ClickNext():
    wait = WebDriverWait(driver, 60)  # 30 seconds wait
    div_locator = (By.XPATH,"/html/body/div[2]/form[2]/div/fieldset/div/div/div[3]/a[3]/span")
    wait.until(EC.element_to_be_clickable(div_locator))

    #Java Click
    element = driver.find_element(By.XPATH, '/html/body/div[2]/form[2]/div/fieldset/div/div/div[3]/a[3]/span')
    driver.execute_script("arguments[0].click();", element)
    
    #Previous Click
    #driver.find_element(By.XPATH,'/html/body/div[2]/form[2]/div/fieldset/div/div/div[3]/a[3]/span').click()
    #print('Click Next')
    return

In [12]:
def waiting():
    div_locator = (By.ID, "j_idt321_content")
    # wait = WebDriverWait(driver, 10)  # 30 seconds wait
    # wait.until(EC.visibility_of_element_located(div_locator))

    wait = WebDriverWait(driver, 60)  # 30 seconds wait
    wait.until(EC.invisibility_of_element_located(div_locator))
    print('Waiting for sandClock Disappear')
    return


In [13]:
def waitingNextPage():
    div_locator = (By.CSS_SELECTOR, ".ui-dialog-content.ui-widget-content")
    
    try:
        wait = WebDriverWait(driver, 3)  # 3 seconds wait
        wait.until(EC.visibility_of_element_located(div_locator))
        print("Waiting icon is visible", end='-->')
    except TimeoutException:
        print("Waiting icon is already visible", end='-->')

    wait = WebDriverWait(driver, 60)  # 30 seconds wait
    div_locator = (By.XPATH, "/html/body/div[9]/div")
    wait.until(EC.invisibility_of_element_located(div_locator))
    print('Waiting icon is invisible')
    return


In [18]:
PartList = ['01431SAET01','01432SAET01','01463S2A010','01463S87A01','01463SAA901','01463SHJA01',
            '01464T0AA02','01464T9AT01','01464TF0010','01465S9A000']

In [14]:
def CheckEmpty():
    waitingNextPage()
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    foundItem = len(soup.find_all('tbody',{'id':'itemSearchTable_data'})[0].find_all('tr'))
    foundText = soup.find_all('tbody',{'id':'itemSearchTable_data'})[0].find_all('tr')[0].text
    if foundText == 'ไม่พบรายการที่ตรงกัน ...':
        return(True)
    else:
        return(False)

In [15]:

def is_next_button_clickable():
    try:
        # Assuming this is the locator for your "next" button
        next_button_locator = (By.XPATH, "/html/body/div[2]/form[2]/div/fieldset/div/div/div[3]/a[3]/span")
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(next_button_locator))

        WebDriverWait(driver, 10).until(EC.visibility_of_element_located(next_button_locator))
        data = driver.page_source
        soup = bs4.BeautifulSoup(data)
        nextButton = len(soup.find_all('a',{'class':'ui-paginator-next ui-state-default ui-corner-all' }))
        if nextButton == 0:
            return False
        else:
            return True         
    except TimeoutException:
        # If the button is not clickable within the timeout period, return False
        return False

In [17]:
def getImage():
    

IndentationError: expected an indented block (3387818342.py, line 2)

In [456]:
ClosePartList()

In [461]:
driver.find_element(By.XPATH,'/html/body/div[2]/form[2]/div/fieldset/div/div/div[1]/button/span[1]').click()

<font color='yellow' size=5>Manual Run</font>

In [96]:
PartID = Hon.iloc[3162:3163, 0].iloc[0]
InputSearchBox(PartID)   

Waiting for sandClock Disappear
input 39794SDA902


In [102]:
#If False
PageNum = ChangeDisplayItem()
print(PageNum)

Waiting icon is already visible-->Waiting icon is invisible
40


In [ ]:
ModelList = pd.concat([ModelList, getModel()], ignore_index=True)

In [ ]:
n = 2
while is_next_button_clickable() and PageNum>1:
    ClickNext()
    waitingNextPage()
    print('Page:' + str(n), end=' ')
    ModelList = pd.concat([ModelList, getModel()], ignore_index=True)
    n += 1

In [105]:
driver.find_element(By.XPATH,'/html/body/div[2]/form[2]/div/fieldset/div/div/div[2]/table/tbody/tr[1]/td[3]/button[2]').click()

In [ ]:
data = driver.page_source
soup = bs4.BeautifulSoup(data)
getIndexOfPartAndClick(soup,PartID)

In [361]:
zoomImage()
CaptureImage(PartID)
DescText = GetDesc()
print(DescText)

Already Pop up
Waiting icon is already visible-->Waiting icon is invisible
done
width=960:height=479
ปลั๊ก หัวเทียน


In [362]:
element = driver.find_element(By.XPATH, '/html/body/div[6]/div/div/div/div[2]/div/div/form[2]/div[2]/div[2]/div/div/div[1]/span/span/button[4]')
driver.execute_script("arguments[0].click();", element)

In [363]:
#Get Cat
driver.execute_script("window.scrollTo(0, 0)")
driver.find_element(By.XPATH,'/html/body/div[6]/div/div/div/div[2]/div/div/form[2]/div[2]/div[1]').click()

In [364]:
data = driver.page_source
soup = bs4.BeautifulSoup(data)
catList = soup.find_all('select',{'name':'catSelection:groupSelect_input'})[0].find_all('option')
cat = [cat for cat in catList if 'selected' in (cat.get('selected') or '')][0].text.replace(',',':')
subCatList = soup.find_all('select',{'name':'catSelection:diagramSelect_input'})[0].find_all('option')
subCat = [subCat for subCat in subCatList if 'selected' in (subCat.get('selected') or '')][0].text.split('\t')[1].strip()
print(cat+':'+subCat)

ENGINE:HIGH TENSION CORD/PLUG


In [365]:
ModelList.loc[ModelList['PartID'].apply(lambda x : x.replace('-',''))==PartID, 'DescText'] = DescText
ModelList.loc[ModelList['PartID'].apply(lambda x : x.replace('-',''))==PartID, 'Cat'] = cat
ModelList.loc[ModelList['PartID'].apply(lambda x : x.replace('-',''))==PartID, 'SubCat'] = subCat

<font color='yellow' size=5>Run</font>

In [400]:
SetupConfig()
#InputSearchBox(PartList[0])
#ChangeDisplayItem()
#ClosePartList()

#ModelList = pd.DataFrame()
for index, row in Hon[15:].iterrows():
    PartID = row['PartNo']
    print(str(index)+':',end='')
    InputSearchBox(PartID)    
    time.sleep(1)  
    print(CheckEmpty())
    
    if CheckEmpty():
        tempTable = pd.DataFrame(columns=['Part', 'PartDescription', 'PartID', 'CarModel','DescText','Cat','SubCat'])
        tempTable.loc[0] = [None,'ไม่พบรายการที่ตรงกัน', PartID, 'ไม่พบรายการที่ตรงกัน', None,None,None]
        ModelList = pd.concat([ModelList, tempTable], ignore_index=True)
        #Close current search
        ClosePartList()
       
    else:
        waitingNextPage()
        PageNum = ChangeDisplayItem()        

        print('Page:1', end=' ')
        ModelList = pd.concat([ModelList, getModel()], ignore_index=True)
        # Start loop
        n = 2
        while is_next_button_clickable() and PageNum>1:
            ClickNext()
            waitingNextPage()
            print('Page:' + str(n), end=' ')
            ModelList = pd.concat([ModelList, getModel()], ignore_index=True)
            n += 1        
        DescText,cat,subCat = getImage(PartID)
        
        ModelList.loc[ModelList['PartID'].apply(lambda x : x.replace('-',''))==PartID, 'DescText'] = DescText
        ModelList.loc[ModelList['PartID'].apply(lambda x : x.replace('-',''))==PartID, 'Cat'] = cat
        ModelList.loc[ModelList['PartID'].apply(lambda x : x.replace('-',''))==PartID, 'SubCat'] = subCat

15:Waiting for sandClock Disappear
input 04602SWA305ZZ
Waiting icon is already visible-->Waiting icon is invisible
True
Waiting icon is already visible-->Waiting icon is invisible
16:Waiting for sandClock Disappear
input 04602SZY310ZZ
Waiting icon is already visible-->Waiting icon is invisible
False
Waiting icon is already visible-->Waiting icon is invisible
Waiting icon is already visible-->Waiting icon is invisible
Waiting icon is already visible-->Waiting icon is invisible
Page:1 Got:0
Waiting icon is already visible-->Waiting icon is invisible
done
width=960:height=479
17:Waiting for sandClock Disappear
input 04602TG0T01ZZ
Waiting icon is already visible-->Waiting icon is invisible
False
Waiting icon is already visible-->Waiting icon is invisible
Waiting icon is already visible-->Waiting icon is invisible
Waiting icon is already visible-->Waiting icon is invisible
Page:1 Got:0
Waiting icon is already visible-->Waiting icon is invisible
done
width=960:height=480
18:Waiting for sandC

KeyboardInterrupt: 

In [397]:
Hon

,PartNo
0,01463S87A01
1,01463SHJA01
2,01464TF0010
3,01465SWA020
4,01465TF0010
...,...
1012,H4502SR3T01
1013,H5699PAAT01
1014,H5699PLMT01
1015,H7662SB3800


In [25]:
ModelList['PartID'].apply(lambda x : x.replace('-',''))#==PartID

Series([], Name: PartID, dtype: object)

In [371]:
ModelList.to_parquet(r'/Users/krichnorakhun/Documents/Machine Learning/selenium/parquet/H9-Table1.parquet')

In [2]:
comb1 = pd.read_parquet(r'/Users/krichnorakhun/Documents/Machine Learning/selenium/parquet/H-Table12.parquet')
comb2 = pd.read_parquet(r'/Users/krichnorakhun/Documents/Machine Learning/selenium/parquet/H2-Table2.parquet')
comb3 = pd.read_parquet(r'/Users/krichnorakhun/Documents/Machine Learning/selenium/parquet/H3-Table1.parquet')
comb4 = pd.read_parquet(r'/Users/krichnorakhun/Documents/Machine Learning/selenium/parquet/H4-Table1.parquet')
comb5 = pd.read_parquet(r'/Users/krichnorakhun/Documents/Machine Learning/selenium/parquet/H5-Table1.parquet')
comb6 = pd.read_parquet(r'/Users/krichnorakhun/Documents/Machine Learning/selenium/parquet/H6-Table1.parquet')
comb7 = pd.read_parquet(r'/Users/krichnorakhun/Documents/Machine Learning/selenium/parquet/H7-Table1.parquet')
comb8 = pd.read_parquet(r'/Users/krichnorakhun/Documents/Machine Learning/selenium/parquet/H8-Table1.parquet')

In [3]:
comb = pd.concat([comb1,comb2,comb3,comb4,comb5,comb6,comb7,comb8], ignore_index=True)
comb = comb.drop_duplicates()
comb = comb.loc[~comb['DescText'].isna()]
comb = comb.drop(['Part'], axis=1)

In [376]:
Hon.head(5)

,PartNo
0,01431SAET01
1,01432SAET01
2,01463S2A010
3,01463S87A01
4,01463SAA901


In [4]:
comb.loc[:,'PartID'] = comb['PartID'].apply(lambda x : x.replace('-',''))

In [8]:
comb = pd.merge(Hon, comb, left_on='PartNo', right_on='PartID', how='left')

In [9]:
Hon = pd.DataFrame(comb.loc[comb['PartID'].isna()]['PartNo'].unique(), columns=['PartNo'])

In [10]:
Hon = Hon.reset_index(drop=True)

In [11]:
comb = comb.loc[~comb['PartID'].isna()].drop_duplicates()
comb = comb.drop(['PartNo'], axis=1)

In [16]:
comb.to_csv(r'/Volumes/500G/GP/Honda/ScrapingResult/Hon_Table_Full_UTF8.csv', index=False , encoding='UTF-8')

In [12]:
comb['PartID'].nunique()

10665

In [78]:
ChangeDisplayItem()

In [16]:
def getIndexOfPartAndClick(soup,PartID):
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    listData = soup.find_all('tbody',{'class':'ui-datatable-data ui-widget-content'})[5].find_all('tr')
    listDataPart = [i.find_all('td')[2].text.replace('หมายเลขอะไหล่','').replace('-','') for i in listData]

    getIndex = listDataPart.index(PartID) if PartID in listDataPart else -1
    selectionID = listData[getIndex].find_all('td')[5].find_all('div')[0].get('id')
    driver.find_element(By.ID,selectionID).click()
    #time.sleep(5)

In [17]:
def zoomImage():
    #Pop image Out and zoom in
    try:
        popUp_locator = (By.XPATH,'/html/body/div[6]/div/div/div/div[2]/div/div/form[2]/div[2]/div[2]/div/div/div[1]/span/span/button[3]')
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located(popUp_locator))
        driver.find_element(*popUp_locator).click()
    except TimeoutException:
        wait = WebDriverWait(driver, 10)
        PopIn = (By.XPATH,'/html/body/div[6]/div/div/div/div[2]/div/div/form[2]/div[2]/div[2]/div/div/div[1]/span/span/button[4]')
        wait.until(EC.visibility_of_element_located(PopIn))
        print('Already Pop up')
        
    while driver.find_element(By.XPATH, '/html/body/div[6]/div/div/div/div[2]/div/div/form[2]/div[2]/div[2]/div/div/div[2]/div/div/div/img[1]').size['height']<450:
        driver.find_element(By.XPATH,'/html/body/div[6]/div/div/div/div[2]/div/div/form[2]/div[2]/div[2]/div/div/div[1]/span/span/button[1]').click()
        time.sleep(1)
    waitingNextPage()

In [18]:
def CaptureImage(PartID):
    # Locate the image element
    img_element = driver.find_element(By.XPATH, '/html/body/div[6]/div/div/div/div[2]/div/div/form[2]/div[2]/div[2]/div/div/div[2]/div/div/div/img[1]')

    # Get the location and size of the image element
    location = img_element.location
    size = img_element.size

    # Take a screenshot of the entire page
    driver.execute_script("window.scrollTo(0, 0)")
    time.sleep(2)
    try:
        driver.save_screenshot('page_screenshot.png')
    except TimeoutException:
        time.sleep(2)
        driver.save_screenshot('page_screenshot.png')
    finally:
        print('done')

    # Use PIL to crop the image
    x = location['x']
    y = location['y']
    width = size['width']
    height = size['height']
    im = Image.open('page_screenshot.png')
    im = im.crop((int(x), int(y), int(x + width), int(y + height)))
    im.save('images/HON-'+PartID+'.png')
    print('width='+str(width)+':height='+str(height))


In [19]:
def GetDesc():
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    listTable = soup.find_all('div',{'class':'ui-tabs-panels'})[0].find_all('tbody',{'id':'lonTabView:lonTable_data'})[0].find_all('tr')
    listTable = [row for row in listTable if 'highlightLON' in row.get('class')]
    listTable = [row.find_all('td')[3].text for row in listTable]
    listTable = [
        row.replace('(เกี่ยวเนื่องกับเลขอ้างอิง)/รายละเอียดค่าแรง', '').split('-')[0].replace(',', '').strip() 
        for row in listTable 
        if any(substring in row for substring in ['ตรวจและซ่อมแซม', 'เปลี่ยน'])
    ]
    DescText = ':'.join(listTable)
    return(DescText)


In [20]:
def getCat():
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    catList = soup.find_all('select',{'name':'catSelection:groupSelect_input'})[0].find_all('option')
    cat = [cat for cat in catList if 'selected' in (cat.get('selected') or '')][0].text.replace(',',':')
    subCatList = soup.find_all('select',{'name':'catSelection:diagramSelect_input'})[0].find_all('option')
    subCat = [subCat for subCat in subCatList if 'selected' in (subCat.get('selected') or '')][0].text.split('\t')[1].strip()
    return(cat, subCat)


In [21]:
def getImage(PartID):
    wait = WebDriverWait(driver, 10)
    picToClick = (By.XPATH, "/html/body/div[2]/form[2]/div/fieldset/div/div/div[2]/table/tbody/tr[1]/td[3]/button[2]")
    wait.until(EC.element_to_be_clickable(picToClick))
    driver.find_element(By.XPATH,'/html/body/div[2]/form[2]/div/fieldset/div/div/div[2]/table/tbody/tr[1]/td[3]/button[2]').click()
    #Waiting for image displayed
    wait = WebDriverWait(driver, 60)  # 60 seconds wait
    image_locator = (By.XPATH, "/html/body/div[6]/div/div/div/div[2]/div/div/form[2]/div[2]/div[2]/div/div/div[2]/div/div/div")
    wait.until(EC.visibility_of_element_located(image_locator))

    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    getIndexOfPartAndClick(soup,PartID)
    zoomImage()
    CaptureImage(PartID)
    DescText = GetDesc()
    #Click Pop In and Close
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    time.sleep(1)
    getIndexOfPartAndClick(soup,PartID) 
    time.sleep(1)
    #Clear checkbox
    wait = WebDriverWait(driver, 10)
    PopIn = (By.XPATH,'/html/body/div[6]/div/div/div/div[2]/div/div/form[2]/div[2]/div[2]/div/div/div[1]/span/span/button[4]')
    wait.until(EC.visibility_of_element_located(PopIn))    
    
    driver.execute_script("window.scrollTo(0, 0)")
    time.sleep(1)
    wait = WebDriverWait(driver, 10)
    PopIn = (By.XPATH,'/html/body/div[6]/div/div/div/div[2]/div/div/form[2]/div[2]/div[2]/div/div/div[1]/span/span/button[4]')    
    wait.until(EC.element_to_be_clickable(PopIn))   
    element = driver.find_element(By.XPATH, '/html/body/div[6]/div/div/div/div[2]/div/div/form[2]/div[2]/div[2]/div/div/div[1]/span/span/button[4]')
    driver.execute_script("arguments[0].click();", element)
    #driver.find_element(*PopIn).click()    
    
    #Click See catetory
    driver.execute_script("window.scrollTo(0, 0)")
    driver.find_element(By.XPATH,'/html/body/div[6]/div/div/div/div[2]/div/div/form[2]/div[2]/div[1]').click()
    
    cat,subCat = getCat()
    #Click Close
    driver.find_element(By.XPATH,'/html/body/div[6]/div/div/div/div[2]/div/div/form[1]/div/ul/li[1]/a').click()
    return(DescText,cat,subCat)


In case of failure because of screen saver.

In [26]:
cat,subCat = getCat()

In [28]:
subCat

'CONTROL UNIT(ENGINE ROOM) (1)'

In [423]:
#In case error because of screen saver.
CaptureImage(PartID)

DescText = GetDesc()
#Click Pop In and Close
data = driver.page_source
soup = bs4.BeautifulSoup(data)
time.sleep(1)
getIndexOfPartAndClick(soup,PartID) 
time.sleep(1)
#Clear checkbox
wait = WebDriverWait(driver, 10)
PopIn = (By.XPATH,'/html/body/div[6]/div/div/div/div[2]/div/div/form[2]/div[2]/div[2]/div/div/div[1]/span/span/button[4]')
wait.until(EC.visibility_of_element_located(PopIn))    
    
driver.execute_script("window.scrollTo(0, 0)")
time.sleep(1)
driver.find_element(*PopIn).click()    
    
#Click See catetory
driver.execute_script("window.scrollTo(0, 0)")
driver.find_element(By.XPATH,'/html/body/div[6]/div/div/div/div[2]/div/div/form[2]/div[2]/div[1]').click()
    
cat,subCat = getCat()
#Click Close
driver.find_element(By.XPATH,'/html/body/div[6]/div/div/div/div[2]/div/div/form[1]/div/ul/li[1]/a').click()

done
width=960:height=480


TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x00000001049e84dc chromedriver + 4162780
1   chromedriver                        0x00000001049e0664 chromedriver + 4130404
2   chromedriver                        0x0000000104637bc0 chromedriver + 293824
3   chromedriver                        0x000000010467e040 chromedriver + 581696
4   chromedriver                        0x00000001046bdab0 chromedriver + 842416
5   chromedriver                        0x00000001046716bc chromedriver + 530108
6   chromedriver                        0x0000000104672930 chromedriver + 534832
7   chromedriver                        0x00000001049ade08 chromedriver + 3923464
8   chromedriver                        0x00000001049b23dc chromedriver + 3941340
9   chromedriver                        0x0000000104996038 chromedriver + 3825720
10  chromedriver                        0x00000001049b2f3c chromedriver + 3944252
11  chromedriver                        0x00000001049886f4 chro

In [32]:
ModelList.loc[ModelList['PartID'].apply(lambda x : x.replace('-',''))==PartID, 'DescText'] = DescText
ModelList.loc[ModelList['PartID'].apply(lambda x : x.replace('-',''))==PartID, 'Cat'] = cat
ModelList.loc[ModelList['PartID'].apply(lambda x : x.replace('-',''))==PartID, 'SubCat'] = subCat

In [ ]:
DescText,cat,subCat = getImage(PartID)
ModelList.loc[ModelList['PartID'].apply(lambda x : x.replace('-',''))==PartID, 'DescText'] = DescText
ModelList.loc[ModelList['PartID'].apply(lambda x : x.replace('-',''))==PartID, 'Cat'] = cat
ModelList.loc[ModelList['PartID'].apply(lambda x : x.replace('-',''))==PartID, 'SubCat'] = subCat

In [116]:
ModelList

,Part,PartDescription,PartID,CarModel,DescText,Cat,SubCat
0,37870-PJ7-003 - 18S04KT1 ( CIVIC 4Dr 1996 TH -...,SENSOR ASSY.: WATER TEMP (TW5) (MATSUSHITA),37870-PJ7-003,CIVIC 4Dr 1996 TH - [ 5MT - LXI ],เซ็นเซอร์ อุณหภูมิน้ำ,ENGINE,WATER PUMP/SENSOR
1,37870-PJ7-003 - 18S04KT1 ( CIVIC 4Dr 1996 TH -...,SENSOR ASSY.: WATER TEMP (TW5) (MATSUSHITA),37870-PJ7-003,CIVIC 4Dr 1996 TH - [ 5MT - LXI ],เซ็นเซอร์ อุณหภูมิน้ำ,ENGINE,WATER PUMP/SENSOR
2,37870-PJ7-003 - 18S04KT1 ( CIVIC 4Dr 1996 TH -...,SENSOR ASSY.: WATER TEMP (TW5) (MATSUSHITA),37870-PJ7-003,CIVIC 4Dr 1996 TH - [ 5MT - LXI ],เซ็นเซอร์ อุณหภูมิน้ำ,ENGINE,WATER PUMP/SENSOR
3,37870-PJ7-003 - 18S04KT1 ( CIVIC 4Dr 1996 TH -...,SENSOR ASSY.: WATER TEMP (TW5) (MATSUSHITA),37870-PJ7-003,CIVIC 4Dr 1996 TH - [ 4AT - EXI ],เซ็นเซอร์ อุณหภูมิน้ำ,ENGINE,WATER PUMP/SENSOR
4,37870-PJ7-003 - 18S04KT1 ( CIVIC 4Dr 1996 TH -...,SENSOR ASSY.: WATER TEMP (TW5) (MATSUSHITA),37870-PJ7-003,CIVIC 4Dr 1996 TH - [ 4AT - EXI ],เซ็นเซอร์ อุณหภูมิน้ำ,ENGINE,WATER PUMP/SENSOR
...,...,...,...,...,...,...,...
11715,39794-SDA-902 - 18TY2D01 ( LEGEND 4Dr 2016 TH ...,RELAY ASSY.: ENGINE CONTROL MODULE (MITSUBA) (...,39794-SDA-902,LEGEND 4Dr 2016 TH - [ DCT - EX ],RELAY MAIN SOLENOID:RELAY IGNITION COIL,ELECTRICAL EQUIPMENTS:EXHAUST:HEATER,CONTROL UNIT(ENGINE ROOM) (2)
11716,39794-SDA-902 - 18TY2D01 ( LEGEND 4Dr 2016 TH ...,RELAY ASSY.: ENGINE CONTROL MODULE (MITSUBA) (...,39794-SDA-902,LEGEND 4Dr 2016 TH - [ DCT - EX ],RELAY MAIN SOLENOID:RELAY IGNITION COIL,ELECTRICAL EQUIPMENTS:EXHAUST:HEATER,CONTROL UNIT(ENGINE ROOM) (2)
11717,39794-SDA-902 - 18TY2D01 ( LEGEND 4Dr 2016 TH ...,RELAY ASSY.: ENGINE CONTROL MODULE (MITSUBA) (...,39794-SDA-902,LEGEND 4Dr 2016 TH - [ DCT - EX ],RELAY MAIN SOLENOID:RELAY IGNITION COIL,ELECTRICAL EQUIPMENTS:EXHAUST:HEATER,CONTROL UNIT(ENGINE ROOM) (2)
11718,39794-SDA-902 - 18TY2D01 ( LEGEND 4Dr 2016 TH ...,RELAY ASSY.: ENGINE CONTROL MODULE (MITSUBA) (...,39794-SDA-902,LEGEND 4Dr 2016 TH - [ DCT - EX ],RELAY MAIN SOLENOID:RELAY IGNITION COIL,ELECTRICAL EQUIPMENTS:EXHAUST:HEATER,CONTROL UNIT(ENGINE ROOM) (2)
